<a href="https://colab.research.google.com/github/hphuocthanh/stock-trading-analysis/blob/master/financial_news_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

True

## Install requirements

In [ ]:
!pip install datasets transformers huggingface_hub accelerate

## Load dataset

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

SEED = 42
train_data = dataset["train"].shuffle(seed=SEED)
validation_data = dataset["validation"].shuffle(seed=SEED)

print(train_data[:3])
print(validation_data[:3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': ['Stocks - Tiffany, Disney, Tesla Rise Premarket; Uber Falls', 'Leafly Jobs Report: Cannabis is the fastest-growing American industry, surpassing 240,000 jobs', 'Added $STNE , $GSX and $BKE to the #PowerEarningsGap list \n\nThese 3 names look very promising going forward! Look f… https://t.co/yABsTNVDIO'], 'label': [2, 1, 1]}
{'text': ['Lupin reports Q3 results', '$ABEO as expected, keeps going higher. Cantor doubled its price target this morning to $4', 'Is Seritage Growth Properties (SRG) A Good Stock To Buy?'], 'label': [2, 1, 2]}


## Bert

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = validation_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Train the model

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels,average='micro')["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-bert-model"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Step,Training Loss
500,0.081300
1000,0.053300


Checkpoint destination directory finetuning-sentiment-bert-model/checkpoint-597 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory finetuning-sentiment-bert-model/checkpoint-1194 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1194, training_loss=0.07148591397795047, metrics={'train_runtime': 115.1309, 'train_samples_per_second': 165.776, 'train_steps_per_second': 10.371, 'total_flos': 254904937650432.0, 'train_loss': 0.07148591397795047, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.6922785043716431,
 'eval_accuracy': 0.873534338358459,
 'eval_f1': 0.873534338358459,
 'eval_runtime': 5.7571,
 'eval_samples_per_second': 414.789,
 'eval_steps_per_second': 26.055,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1708123370.eb37d45cdc22.3278.3:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hphuocthanh/finetuning-sentiment-bert-model/commit/a100b7b1705af13fef5638efc771a89a591ea646', commit_message='End of training', commit_description='', oid='a100b7b1705af13fef5638efc771a89a591ea646', pr_url=None, pr_revision=None, pr_num=None)

### Test model

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="hphuocthanh/finetuning-sentiment-bert-model")
sentiment_model(["Acorns CEO: Parents can turn $5 into five figures for their kids through 'power of compounding'", "Jim Cramer breaks down his favorite stocks from a record second quarter"])

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_2', 'score': 0.9996703863143921},
 {'label': 'LABEL_2', 'score': 0.999664306640625}]